In [28]:
# Import libraries
import os
import pandas as pd
import re
import json
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter

In [8]:
with open('/kaggle/input/uit-visquad/visquad_train.json', 'r', encoding='utf-8') as f:
    input_data = json.load(f)

In [9]:
# Khởi tạo danh sách để chứa các mục dữ liệu đã làm phẳng
flattened_data = []

# Duyệt qua các mục dữ liệu trong danh sách 'data'
for item in input_data['data']:
    title = item['title']
    for paragraph in item['paragraphs']:
        paragraphs = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            question_id = qa['id']
            is_impossible = qa['is_impossible']
            # Giả sử mỗi câu hỏi chỉ có một câu trả lời chính thức và một câu trả lời hợp lý, nếu có nhiều, bạn cần điều chỉnh mã
            answer = qa['answers'][0]['text'] if qa['answers'] else None
            plausible_answer = qa['plausible_answers'][0]['text'] if 'plausible_answers' in qa and qa['plausible_answers'] else None
            
            # Thêm mục dữ liệu đã làm phẳng vào danh sách
            flattened_data.append({
                'title': title,
                'paragraphs': paragraphs,
                'question': question,
                'question_id': question_id,
                'is_impossible': is_impossible,
                'answer': answer,
                'plausible_answer': plausible_answer
            })

In [10]:
uit_df = pd.DataFrame(flattened_data)

# Hiển thị các dòng đầu tiên của DataFrame
uit_df.head()

,title,paragraphs,question,question_id,is_impossible,answer,plausible_answer
0,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4...,Tên gọi nào được Phạm Văn Đồng sử dụng khi làm...,uit_000001,False,Lâm Bá Kiệt,None
1,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4...,Phạm Văn Đồng giữ chức vụ gì trong bộ máy Nhà ...,uit_000002,False,Thủ tướng,None
2,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4...,"Giai đoạn năm 1955-1976, Phạm Văn Đồng nắm giữ...",uit_000003,False,Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa,None
3,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4...,Tên gọi nào được Phạm Văn Đồng sử dụng trước k...,uit_000004,True,None,Lâm Bá Kiệt
4,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4...,Hồ Học Lãm giữ chức vụ gì trong bộ máy Nhà nướ...,uit_000005,True,None,Thủ tướng


In [11]:
print(uit_df['is_impossible'].value_counts())

is_impossible
False    19240
True      9217
Name: count, dtype: int64


In [22]:
count_is_impossible_true = uit_df[uit_df['is_impossible'] == True].shape[0]
count_is_impossible_false = uit_df[uit_df['is_impossible'] == False].shape[0]
# Đếm số hàng có giá trị trong cột plausible_answer (không phải NaN)
count_plausible_answer = uit_df['plausible_answer'].notna().sum()
count_answer = uit_df['answer'].notna().sum()
# In ra kết quả đếm để kiểm tra
print("Số lượng hàng với is_impossible = True:", count_is_impossible_true)
print("Số lượng hàng với is_impossible = False:", count_is_impossible_false)
print("Số lượng hàng có plausible_answer:", count_plausible_answer)
print("Số lượng hàng có answer:", count_answer)

Số lượng hàng với is_impossible = True: 9217
Số lượng hàng với is_impossible = False: 19240
Số lượng hàng có plausible_answer: 9217
Số lượng hàng có answer: 19240


In [23]:
uit_df.isnull().sum()

title                   0
paragraphs              0
question                0
question_id             0
is_impossible           0
answer               9217
plausible_answer    19240
dtype: int64

In [13]:
# Load the data from train.json
with open('/kaggle/input/traintest/train.json', 'r', encoding='utf-8') as file:
    zalo_train_data = json.load(file)


In [14]:
# Khởi tạo danh sách để chứa các mục dữ liệu
flattened_zalo_data = []

# Duyệt qua các mục dữ liệu trong 'root'
for item in zalo_train_data:
    # Thêm mục dữ liệu vào danh sách
    flattened_zalo_data.append({
        'id': item['id'],
        'question': item['question'],
        'title': item['title'],
        'text': item['text'],
        'label': item['label']
    })

In [15]:
# Tạo DataFrame từ danh sách các từ điển
zalo_df = pd.DataFrame(flattened_zalo_data)

# Hiển thị các dòng đầu tiên của DataFrame
zalo_df.head()

,id,question,title,text,label
0,u7-1570446247_1,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),"Năm 2013 , Nguyễn Quang Hải giành chức vô địch...",True
1,u7-1570446247_2,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),"Sau chức vô địch U-21 quốc gia 2013 , Nguyễn Q...",True
2,u7-1570446247_0,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),Anh bắt đầu gia nhập lò đào tạo trẻ Hà Nội T&T...,False
3,u7-1570446247_3,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),"Năm 2011 , Nguyễn Quang Hải mới 14 tuổi được g...",False
4,u7-1570445661_0,Mỗi hiệp bóng đá kéo dài bao lâu,Bóng đá,Một trận đấu bóng đá thông thường có hai hiệp ...,True


In [16]:
print(zalo_df['label'].value_counts())

label
False    12370
True      5738
Name: count, dtype: int64


In [24]:
zalo_df.isnull().sum()

id          0
question    0
title       0
text        0
label       0
dtype: int64

In [17]:
# Load the data from train.json
with open('/kaggle/input/traintest/test.json', 'r', encoding='utf-8') as file:
    zalo_test_data = json.load(file)

In [18]:
# Khởi tạo danh sách để chứa các mục dữ liệu
flattened_test_data = []

# Duyệt qua các mục dữ liệu trong 'root'
for item in zalo_test_data:
     # Duyệt qua các đoạn văn trong mỗi mục
    for paragraph in item['paragraphs']:
        # Thêm mục dữ liệu vào danh sách
        flattened_test_data.append({
            'id': item['__id__'],
            'question': item['question'],
            'title': item['title'],
            'paragraph_id': paragraph['id'],
            'text': paragraph['text']
        })

In [19]:
# Tạo DataFrame từ danh sách các từ điển
test_df = pd.DataFrame(flattened_test_data)

# Hiển thị các dòng đầu tiên của DataFrame
test_df.head()

,id,question,title,paragraph_id,text
0,test_0001,Toà nhà nào cao nhất Việt Nam,The Landmark 81,p1,The Landmark 81 là một toà nhà chọc trời trong...
1,test_0001,Toà nhà nào cao nhất Việt Nam,The Landmark 81,p2,"Toà tháp cao 81 tầng , hiện tại là toà nhà cao..."
2,test_0002,Messi thuận chân nào,Lionel Messi,p1,Đôi chân ngắn nhưng mạnh mẽ giúp Messi dễ dàng...
3,test_0002,Messi thuận chân nào,Lionel Messi,p2,"Cựu huấn luyện viên Barcelona , Pep Guardiola ..."
4,test_0002,Messi thuận chân nào,Lionel Messi,p3,Ngoài ra anh còn một chân sút phạt chính và ha...


In [20]:
num_rows = test_df.shape[0]
print(f"Số hàng trong test_df: {num_rows}")

Số hàng trong test_df: 2678


In [25]:
test_df.isnull().sum()

id              0
question        0
title           0
paragraph_id    0
text            0
dtype: int64